### Created all parts as functions to allow to be run multiple times with varied parameters

### Import Required Modules 

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

import scipy.optimize as opt

#%matplotlib qt
# qt if plot in seperate window
# inline if plot on page

## Set the number of d.p. displayed in numpy arrays
np.set_printoptions(precision=3)

## Progress bar widget
from IPython.html.widgets import FloatProgress
from IPython.display import display

## allow timing of events
import time

## allow export of results as csv
import csv

In [2]:
### Alpha beta calculator - normal dist

def alphacalc_normal(alphabeta, sd):
    """Return alphabetanew and alpha from normal distribution as specified by sd.
    Default is beta = 0.03
    'alphabeta' is the alphabeta ratio
    If a negative value is returned it is resampled until positive"""
    
    beta = 0.03 # fixed beta in function
    
    ## get alpha beta to use from normal distribution
    if sd == 0:
        alphabetanew = alphabeta
    else:
        alphabetanew=np.random.normal(loc = alphabeta, scale = sd)
    
    ## make sure a positive value is returned
    while alphabetanew <= 0:
        alphabetanew=np.random.normal(loc = alphabeta, scale = sd)
    
    alpha = beta*alphabetanew
   
    return alpha, beta
## alpha/beta can be calced form the returned alpha and beta values

In [3]:
### Alpha beta calculator - log normal dist

def alphacalc_lognormal(alphabeta, sd):
    """Return alphabetanew and alpha from normal distribution as specified by sd.
    Default is beta = 0.03
    'alphabeta' is the alphabeta ratio"""
    
    beta = 0.02 # fixed beta in function
    
    alphabeta_lognormal = np.log((alphabeta**2)/(np.sqrt((sd**2)+(alphabeta**2))))
    sd_lognormal = np.sqrt(np.log(((sd**2)/(alphabeta**2))+1))
    
    ## get alpha beta to use from normal distribution
    if sd == 0:
        alphabetanew = alphabeta
    else:
        alphabetanew=np.random.lognormal(mean = alphabeta_lognormal, sigma = sd_lognormal)
    
    alpha = beta*alphabetanew
   
    return alpha, beta
## alpha/beta can be calced form the returned alpha and beta values

###N0 varies depending on the a/b Std Dev!!!

Need to rethink how to determine N0 in a simple way.

- Ideally re-write entire thing to accept a set of parameters and then use these single parameters to feed into other functions.


**Ideas**
Could use some sort of fitting with all parameters fixed except N0.
Then try and minimise the difference between TCP_calc and TCP_input?
- Can set it to repeat the TCP calc for 1000 patients, then returnt he TCP at d_interest.
- This would use just a single set of nominal parameters (as input by user/in model).
- Only after this does the simulation of multiple parameter variations start.

I cannot think of another way of doing this, as I ahve been doing it manually (trial and error) so far...


In [189]:
## Determine N0 by minimising difference betwen calculate TCP and input TCP.

TCP_input = 60 # TCP in percent

## Return sum of squares [calc = calculation of TCP from TCP calc, ref = TCP_input]
def sq_dif(calc, ref):
    sq_dif = (calc - ref)**2
    return sq_dif

### Need to add something to ensure the minimum is always found, but without always having a huge list of possible values.
## Could also start with a coarse list and refine after one sweep?
j=1
start = 0
increment=10

any_pos = False
any_neg = False
stop = False

while (stop == False and j<100):
    TCP_list = []
       
    a_list = list(range(start,(j*increment)))
    print(a_list)
    
    for i in a_list:
        TCP_Calc = completeTCPcalc(n=100,
                               alphabeta_use=3,
                               alphabeta_sd_use=1,
                               d=2,
                               d_shift=0,
                               d_sd=1,
                               n0=i, # this needs to be able to vary to allow optimisation
                               max_d=100,
                               dose_of_interest=74)
        TCP_list.append(TCP_Calc[10])

    dif_list = np.array(TCP_list)-TCP_input
    dif_list_abs = abs(np.array(TCP_list)-TCP_input)
    print("test")
    print(dif_list)

    any_pos = np.any(dif_list >0)
    any_neg = np.any(dif_list <0)
    
    if (any_pos == True and any_neg == True):
        stop = True
        #break
    
    j = j+1 # increment list to lookup
    start = start + increment
    
print("N0 fitted")
## can use the following to check if the minimum has been found by cehckign for + and - differences.


#dif_list[:] = [x-TCP_input for x in TCP_list]

#a[:] = [x - 13 for x in a]
    
### Now want to 

#tester = sp.optimize.minimize(sq_dif(TCP_Calc(x),TCP_input))
print(min(dif_list_abs)) # minimum difference found.
print(np.argmin(dif_list_abs))# index of minimum found
print("N0: " + str(a_list[np.argmin(dif_list_abs)]))

## So n0 = a_list[np.argmin(dif_list)]

TCP_Calc_min = completeTCPcalc(n=1000,
                           alphabeta_use=3,
                           alphabeta_sd_use=1,
                           d=2,
                           d_shift=0,
                           d_sd=1,
                           n0=a_list[np.argmin(dif_list_abs)], # this needs to be able to vary to allow optimisation
                           max_d=100,
                           dose_of_interest=74)
print(TCP_Calc_min[10])


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
test
[ 40.     39.885  39.738  39.61   39.419  39.393  39.257  39.042  38.889
  39.095]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
test
[ 38.638  38.586  38.291  38.131  38.248  37.915  38.321  37.71   37.653
  37.754]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
test
[ 37.423  37.391  37.35   37.437  36.841  37.009  36.588  36.096  36.069
  36.062]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
test
[ 36.508  35.575  35.413  36.161  35.493  35.741  35.489  34.866  35.729
  33.929]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
test
[ 34.718  35.315  35.104  34.706  34.569  33.994  35.341  34.742  35.
  33.269]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
test
[ 33.791  34.344  32.941  32.885  33.92   33.043  33.578  33.616  33.77
  32.675]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
test
[ 31.763  32.18   31.989  31.863  31.62   32.355  30.939  31.967  31.599
  31.624]
[70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
test
[ 31.327  32.271  30.021  31.574  32.272  31.645  31.45   31.142 

In [346]:
## function to calculate the difference between input TCP and calculated TCP. only N0 is varied as want to estimate this.
def calc_dif_sq(x,*TCP):
    TCP_in = TCP
    
    TCP_calc_all = completeTCPcalc(n=2000,
                               alphabeta_use=3,
                               alphabeta_sd_use=1,
                               d=2,
                               d_shift=0,
                               d_sd=1,
                               n0=x, # this needs to be able to vary to allow optimisation of the value
                               max_d=100,
                               dose_of_interest=74)
    TCP_result = TCP_calc_all[10] ## Get only the result of interest (N0 is in posn. 10 int he returned tuple)
    
    TCP_Dif_sq = (TCP_in - TCP_result)**2 ## difference in squares to minimise
    return TCP_Dif_sq

In [347]:
## Determine N0 by minimising difference betwen calculate TCP and input TCP.

## TCP to fit N0 to. This will come from the literature.
TCP_input = 65

## Run optimisation multiple times to ensure accuracy (as based on random data)
repeats = 50
## store the fit results in this list
fit_results=[]

for i in range(0,repeats):
    N0_result = opt.minimize_scalar(calc_dif_sq,method='brent',args=(TCP_input))
    fit_results.append(N0_result.x[0]) ## build up a list of the N0 fit results
    no_fit_perc_completed = 100*i/repeats  ## Display the progress
    print("Fitting N0: " + str(int(no_fit_perc_completed)) + "% complete", end="\r")
    
print("Fitting N0: 100% complete")

## Remove outliers from N0 predictions (based on keeping the central 30% of results (within ~0.5 SD))
trimmed = fit_results[:]
while len(trimmed)>0.3*len(fit_results):
    trimmed.remove(max(trimmed))
    trimmed.remove(min(trimmed))

## Calculate mean of remaining results
N0_mean_fit=sum(trimmed)/len(trimmed)
print("N0 mean fit: " + str(N0_mean_fit))

## Perform check calculation
TCP_Calc_min = completeTCPcalc(n=50000,
                           alphabeta_use=3,
                           alphabeta_sd_use=1,
                           d=2,
                           d_shift=0,
                           d_sd=1,
                           n0=N0_mean_fit, # this needs to be able to vary to allow optimisation
                           max_d=100,
                           dose_of_interest=74)
print('TCP target: ' + str(TCP_input))
print('TCP fit check: ' + str(TCP_Calc_min[10]))

Fitting N0: 100% complete
N0 mean fit: 446.143404938
TCP target: 65
TCP fit check: 65.1468498328


In [348]:
for i in range(5):
    TCP_Calc_min = completeTCPcalc(n=50000,
                               alphabeta_use=3,
                               alphabeta_sd_use=1,
                               d=2,
                               d_shift=0,
                               d_sd=1,
                               n0=N0_mean_fit, # this needs to be able to vary to allow optimisation
                               max_d=100,
                               dose_of_interest=74)
    print(TCP_Calc_min[10])

64.9320412719
65.2626169434
65.2199295191
65.0867840359
65.2282218781


In [368]:
import matplotlib.pyplot as plt

plt.hist(fit_results)
plt.hist(trimmed)
plt.show()

### a/b SD
This parameter makes a significant difference to the N0 and the TCP values

In [4]:
## N0 determination from entered TCP model parameters
def n0_determination(TCP,d=2,D=74,ab=3,b=0.02):
    """Determine the value of N0 from the input of the
    nominal TCP model input parameters.
    This enables the model to match that of clinical trials data if input."""
    a = b * ab #ab ratio = a/b
    n0 = -np.log(TCP)/np.exp(-(a*D)-(b*d*D))
    return n0

In [5]:
abc = n0_determination(0.88,2,74,3,0.02)
print(abc)

209.133405424


In [6]:
tester = np.array([])
tester2 = np.array([])

n2=10000

mean_n = 10
sd_n = 2

### mean and sd required for lognormal calc as described here: http://uk.mathworks.com/help/stats/lognstat.html?refresh=true
mean_l = np.log((mean_n**2)/(np.sqrt((sd_n**2)+(mean_n**2))))
sd_l = np.sqrt(np.log(((sd_n**2)/(mean_n**2))+1))

for i in range(0,n2):
    tester = np.append(tester,[np.random.normal(mean_n,sd_n)])
    tester2 = np.append(tester2,[np.random.lognormal(mean_l,sd_l)])
    
#tester = np.reshape(tester,(n2,12))
#tester2 = np.reshape(tester2,(n2,2))
#print(tester)

print(sd_l)

#plt.hist(tester, bins=30, color='red', alpha=0.5, label='norm')
#plt.hist(tester2, bins=30, color='blue', alpha=0.5, label='lognorm')
#plt.legend()
##print(tester2)
#plt.show()

0.198042200435


In [7]:
###calculate dose for a given fraction based on normal distribution around dose shift
### Log normal distribution not necessary as SD small wrt mean so v unlikely to get negative values returned.
### Could also set a limit on the range (say to +/- 5%)? But wont make much difference with a small SD anyway.

def fracdose(dose, shift, sd):
    """Return dose_actual from normal distribution around dose (Gy) as specified by sd (%) and shift (%).
    Default is dose = 2Gy, shift = 0%, and sd of 0%
    If a negative value is returned it is resampled until positive
    The standard deviation is of the nominal dose"""
    
    ## get actual dose to use from normal distribution based on shift
    
    dose_shift = dose + (dose*shift/100)
    
    ## if sd is zero, then no change to dose
    if sd == 0:
        dose_actual = dose_shift
        return dose_actual
    
    dose_actual=np.random.normal(loc = dose_shift, scale = (dose*sd/100))
    
    ## make sure a positive value is returned
    while dose_actual <= 0:
        dose_actual=np.random.normal(loc = dose_shift, scale = (dose*sd/100))
    
    return dose_actual

In [8]:
## Survival Fraction Calculation
def SFcalc(alpha, beta, dose):
    """Return the SF with input values.
    Note this is for a single dose delivery.
    The product of multiple fractions shoudld be taken
    to give overall SF"""
    
    SF = np.exp(-(alpha*dose) - (beta*(dose**2)))
    
    return SF

In [9]:
## TCP Calculation absed on cumulative SF
def TCPcalc(sf, n0):
    """Return the TCP with input values.
    Based on cumulative SF and N0"""
    
    TCP = np.exp(-n0*sf)
    
    return TCP

In [10]:
## Calc Number of fractions to get to max dose (note: round up as always want an integer)

def no_frac_nom_doses_array(max_d, d):
    n_frac = np.ceil(max_d/d)

    fractions = np.arange(1,n_frac+1)
    #print(fractions)
    nom_doses = np.arange(d,(d*n_frac)+d, step = d)
    #print(nom_doses)
    return fractions, nom_doses, n_frac

In [11]:
## This gives a column with the patient number and makes it easier to check values as going

def create_patients(n):
    
    if n<1:
        n=1
    patients = np.arange(0,n)+1
    patients.shape=(n,1)
    #print(patients)
    return patients

In [12]:
## empty array to store alpha values in (log normal distribution to be used)

def create_alpha_beta_array(n, alphabeta_use, alphabeta_sd_use):
    alpha_and_beta =[]

    for p in range(0,n):
        #alpha_and_beta = np.append(alpha_and_beta,[alphacalc_normal(alphabeta = alphabeta_use, sd=alphabeta_sd_use)])
        alpha_and_beta.append([alphacalc_lognormal(alphabeta = alphabeta_use, sd=alphabeta_sd_use)])

    ## reshape to get a row per patient
    alpha_and_beta_np = np.array(alpha_and_beta)
    alpha_and_beta_np = np.reshape(alpha_and_beta_np,(n,2))
    #print(alpha_and_beta)
    return alpha_and_beta_np

In [13]:
## Calculate Doses for all patients and all fractions and put in an array

def doses_array(n, n_frac, d, d_shift, d_sd):
    doses = []
    for i in range(0,int(n*n_frac)):
        doses.append(fracdose(dose = d, shift=d_shift, sd=d_sd))
    doses_np = np.array(doses)
    doses_np = np.reshape(doses_np,(n,n_frac))
    #print(doses)
    return doses_np

In [14]:
## Combine all results into single array which may be easier to work with for analysis

def combine_results(patients, alpha_and_beta, doses):
    results_wanted = (patients,alpha_and_beta, doses)
    all_results = np.concatenate(results_wanted, axis=1)
    #print(all_results)
    return all_results

In [15]:
## Loop through the doses of the first patient (first row [0] of array)
def calc_all_SFs(patients, n, n_frac, alpha_and_beta, doses):
    SFs = []

    for i in range(0,len(patients)): # loop through each patient (row)
        for j in range(0,int(n_frac)): # loop through each fraction for each patient (col)
            SFs.append(SFcalc(alpha_and_beta[i][0],alpha_and_beta[i][1],doses[i,j]))

    SFs_np = np.array(SFs)
    SFs_np = np.reshape(SFs_np,(n,n_frac))

    ## GEt cumulative SF for each patient
    SF_cum = np.cumprod(SFs_np, axis=1)
    return SFs_np, SF_cum

In [16]:
## append results to text file.

def saveasCSV(filename, array):

    fl = open(filename, 'a', newline='\n')

    writer = csv.writer(fl)
    writer.writerow(array)

    fl.close()

In [17]:
## Calc Number of fractions and nominal dose per fraction to get to max dose
    
def completeTCPcalc(n,
                   alphabeta_use,
                   alphabeta_sd_use,
                   d,
                   d_shift,
                   d_sd,
                   n0,
                   max_d,
                   dose_of_interest):
    
    fractions, nom_doses, n_frac = no_frac_nom_doses_array(max_d,d)

    ## create array containing number of patients in population
    patients = create_patients(n)

    ## Creat array of alpha and veta values for each patient
    alpha_and_beta = create_alpha_beta_array(n, alphabeta_use, alphabeta_sd_use)

    ## array of doses after each fraction for each patient
    doses = doses_array(n, n_frac, d, d_shift, d_sd)

    ## put all results in an array with a patient on each row
    all_results = combine_results(patients, alpha_and_beta, doses)

    ## Calc cumulative SF for all patients (also return individual fraction SFs)
    SFs, SF_cum = calc_all_SFs(patients, n, n_frac, alpha_and_beta, doses)

    ## Calculate TCP for all individual patients and fractions
    TCPs = TCPcalc(sf = SF_cum, n0=n0)

    ## Calculate population TCP by averaging down the columns
    TCP_pop = np.mean(TCPs, axis = 0)

    frac_of_interest = dose_of_interest/d

    TCP_at_dose_of_interest = TCP_pop[frac_of_interest]

    #t_end = time.time()

    #t_total = t_end-t_start
    #print(str(round(t_total,2)) + ' secs for ' + str(n) + ' patients')

    TCPs_of_interest = TCPs[:,frac_of_interest-1]

    TCP_cure = (TCPs_of_interest).sum()
    TCP_cure_percent = 100*TCP_cure/n
    
    return n,alphabeta_use,alphabeta_sd_use,d,d_shift,d_sd,n0,max_d,dose_of_interest,frac_of_interest,TCP_cure_percent, TCPs, TCP_pop, nom_doses

In [18]:
## Plot of individual and population TCPs as a function for ease

def TCP_plot(no_ind_plots, label):
    #no_ind_plots = 50

    ## individual plots cannot be more than total patients
    if(no_ind_plots>n):
        no_ind_plots=n

    ## want to select the individual plots randomly from those calcualted...
    ind_plots = np.random.choice(len(TCPs),no_ind_plots, replace=False)

    ## individuals (specified number of plots chosen)
    for i in ind_plots:
        plt.plot(nom_doses,TCPs[i], color = 'grey', alpha = 0.5)
    ## population
    plt.plot(nom_doses,TCP_pop, color='black', linewidth='2', alpha=0.5)
    plt.plot(nom_doses,TCP_pop, marker = 'o', ls='none', label=label)

    ## plot formatting
    plt.xlim(0,max(nom_doses))
    plt.ylim(0,1.0)
    plt.xlabel('Dose (Gy)')
    plt.ylabel('TCP')
    plt.title('TCPs')
    #plt.legend(loc = 'best', fontsize = 'medium', framealpha = 1)
    plt.axvline(d_interest, color = 'black', ls='--',)
    plt.axhline(TCP_pop[frac_interest-1], color='black', ls='--')

    ## add labels with TCP at dose of interest
    text_string = ('Pop. TCP = ' + str(round(TCP_cure_at_d_interest,2)) + ' % at ' + str(d_interest) + 'Gy')
    plt.text(5,0.4,text_string, backgroundcolor='white')
    plt.legend(loc = 'lower left',numpoints=1)

    plt.show()

### Calcualte and Plot results of single set of parameters

In [21]:
### Individual calculation of TCP at a set value of dose

n_rpts = 1

print("Enter Dose Shift (%)")
applied_shift = float(input())

text_string = ("Shift: " + str(applied_shift) + "%")

for i in range(0,n_rpts):
    t = completeTCPcalc(n = 1000,
                    alphabeta_use = 3,
                    alphabeta_sd_use = 2,
                    d = 2,
                    d_shift = applied_shift, # +1.6,-2.1% from OP data
                    d_sd = 0,
                    n0 = 100,
                    max_d = 100,
                    dose_of_interest = 74)

    n=t[0]
    TCP_pop = t[-2]
    TCPs = t[-3]
    nom_doses = t[-1]
    d_interest = t[8]
    frac_interest = t[9]
    TCP_cure_at_d_interest = t[10]



    #print(TCP_pop)
    TCP_plot(0,text_string)

Enter Dose Shift (%)
0


### User iterator to build list to vary multiple parameters in one go

In [22]:
## Round to nearest n (e.g. nearest 50 n=50)
def round_n(x,n):
    return int(round(x / n)) * n

In [23]:
## will need to iterate over all the values and add them to a list.

## This allows simple building of a tuple containing all possible combinations of values
import itertools

## dose shifts to test
dose_var = 0.5 # dose step size
dose_max = 3
dose_min = -dose_max
dose_number = (dose_max-dose_min)/dose_var+1 # number of points

dose_vals = np.linspace(dose_min,dose_max,dose_number) # dose values to test

## alphabeta values to test
ab_var = 0.5 # step size
ab_max = 4
ab_min = 2
ab_number = (ab_max-ab_min)/ab_var+1 # number of points
ab_vals = np.linspace(ab_min,ab_max,ab_number) # different a/b values to test

## dose of interest to test
di_var = 1 # dose step size
di_max = 74
di_min = 74
di_number = (di_max-di_min)/di_var+1 # number of points
di_vals = np.linspace(di_min,di_max,di_number)

## N0 values - determined from TCP model paramters input (from clinical data - coudl get form text file etc?)
n0_nominal = n0_determination(TCP=0.88,d=2,D=74,ab=3,b=0.02)
n0_var = 10
n0_range = 5 #percentage variation
n0_min = round_n(n0_nominal * (1-n0_range/100),n0_var)
n0_max = round_n(n0_nominal * (1+n0_range/100),n0_var)
n0_number = (n0_max-n0_min)/n0_var+1
n0_vals = np.linspace(n0_min,n0_max,n0_number)

total_tests = len(dose_vals)*len(ab_vals)*len(di_vals)*len(n0_vals)
test_val_iterator = itertools.product(dose_vals,ab_vals,di_vals,n0_vals)

test_vals = list(test_val_iterator) #list of all combinations of parameters
num_its = len(test_vals) #number of iterations
    
print(num_its)
print(n0_vals)

195
[ 200.  210.  220.]


In [24]:
from datetime import datetime

In [25]:
## vary multiple values through use of constructer iterator list above.

t1 = datetime.now()

k = 10 # number of repeats

f = FloatProgress(min=0, max=num_its)
display(f)

f1 = FloatProgress(min=0, max=k-1)
display(f1)

barpos = 1

all_test_results_array = np.array([]) # array to store all results in before saving as excel csv file
all_test_results_array = []

for j in test_vals:
    #results_array = np.array([])
    results_array = []
    
    for i in range(0,k):
        t = completeTCPcalc(n = 50,
                        alphabeta_use = j[1],
                        alphabeta_sd_use = 2,
                        d = 2,
                        d_shift = j[0],
                        d_sd = 1,
                        n0 = j[3],
                        max_d = 100,
                        dose_of_interest = j[2])
        results_array.append(TCP_cure_at_d_interest)
        f1.value = i    

        n=t[0]
        alphabeta_use = t[1]
        alphabeta_sd_use = t[2]
        d = t[3]
        d_shift = t[4]
        d_sd = t[5]
        n0 = t[6]
        TCP_pop = t[-2]
        TCPs = t[-3]
        nom_doses = t[-1]
        d_interest = t[8]
        frac_interest = t[9]
        TCP_cure_at_d_interest = t[10]
        max_d = 100

        param_array = []
        param_array.append(n)
        param_array.append(k)
        param_array.append(alphabeta_use)
        param_array.append(alphabeta_sd_use)
        param_array.append(d)
        param_array.append(d_shift)
        param_array.append(d_sd)
        param_array.append(n0)
        param_array.append(max_d)
        param_array.append(d_interest)

    ## Array containing results form a single set of parameters (this will update each iteration)
    param_array.extend(results_array)
    param_results_array = param_array
    
    ## Create an array containing all the results for all sets of parameters
    all_test_results_array.extend(param_results_array)
    
    # for updating progress bar
    barpos = barpos+1
    f.value = barpos

#print(param_results_array[-1])

## Covnert to Numpy array and reshape so a single set of parameters is on one row
all_test_results_array_np = np.array(all_test_results_array)
all_test_results_array_np = np.reshape(all_test_results_array_np,(num_its,len(all_test_results_array_np)/num_its))

## Save results to file
np.savetxt("TCPresultsProst-all_results1.csv", all_test_results_array_np, delimiter=",",fmt='%10.3f')
print("Completed")

t2 = datetime.now()
t3 = (t2-t1).total_seconds()

print(t3)

Completed
38.649332
